In [ ]:
#%matplotlib agg
%matplotlib inline
import os
import uproot
import ROOT as r
import math
import awkward as ak
import hist
from hist import Hist
import sys
import argparse
import hist
from hist import loc
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep
mpl.style.use(mplhep.style.ROOT)
from matplotlib.colors import LinearSegmentedColormap
from SimpEquations_validated import SimpEquations as simpeqs

# Define the custom colormap
cmap = LinearSegmentedColormap.from_list('custom_cmap', ['white', 'blue'])
colors = [(1, 1, 1), (0, 0, 0.5), (0.5, 0.5, 0), (1, 0, 0)]  # White, Dark Blue, Yellow, Red
n_bins = [0, 0.00001, 0.5, 1.0]
cmap = LinearSegmentedColormap.from_list('custom_cmap', list(zip(n_bins, colors)))



In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-m', '--mass',help='mass in mev', default=55.0)
parser.add_argument('-c','--cutVariables', type=str, dest="cutVariables",
                    help='Select cut variables to iterate', default=[],nargs='*')
args = parser.parse_args(args=[])

## **Define useful functions**

In [ ]:
def getTBranchArrays(events, branches=[],mass_low=0.0, mass_high=999.9):
    #arrays = events.arrays(branches, f'unc_vtx_mass*1000. > {mass_low} & unc_vtx_mass*1000. < {mass_high}')
    #arrays = events.arrays(branches, "unc_vtx_mass*1000. > 52.0  & unc_vtx_mass*1000. < 57.0")
    arrays = events.arrays(branches,f"(unc_vtx_mass*1000.0 > {mass_low}) & (unc_vtx_mass*1000.0 < {mass_high})")
    return arrays

def applyCorrection(array, variable, correction):
    array[variable] = array[variable] + correction
    
def v0ProjSigCut(array, cut_value):
    condition = array["unc_vtx_proj_sig"] < cut_value
    array = array[condition]
    return array
    
def deltaZCut(array, par0, par1, mass_mev):
    cut = par0 + par1*mass_mev
    condition = array["unc_vtx_deltaZ"] < cut
    array = array[condition]
    return array

def zalphaCut(array, cut_value):
    condition = array["unc_vtx_zalpha_max"] < cut_value
    array = array[condition]
    return array

def calculateZBi(n_on, n_off, tau):
    P_Bi = r.TMath.BetaIncomplete(1./(1.+tau),n_on,n_off+1) #why plus 1?
    Z_Bi = 2.**(0.5)*r.TMath.ErfInverse(1-2*P_Bi)
    return Z_Bi
    

def zalphaTransformation(slope, recon_z, z0):
    condition = z0 > 0.0
    zalpha = ak.where(condition, recon_z - (z0/slope), recon_z + (z0/slope))
    return zalpha

## **Define RadFrac, RadAcc, MassRes**

In [ ]:
def radiativeFraction(mass_mev):
    radF = -1.04206e-01 + 9.92547e-03*mass_mev + -1.99437e-04*pow(mass_mev,2) + 1.83534e-06*pow(mass_mev,3) + -7.93138e-9*pow(mass_mev,4) + 1.30456e-11*pow(mass_mev,5) #alic 2016 simps kf 11/15/22
    return radF

def radiativeAcceptance(mass_mev):
    acc = ( -7.35934e-01 + 9.75402e-02*mass_mev + -5.22599e-03*pow(mass_mev,2) + 1.47226e-04*pow(mass_mev,3) + -2.41435e-06*pow(mass_mev,4) + 2.45015e-08*pow(mass_mev,5) + -1.56938e-10*pow(mass_mev,6) + 6.19494e-13*pow(mass_mev,7) + -1.37780e-15*pow(mass_mev,8) + 1.32155e-18*pow(mass_mev,9) ) #alic 2016 simps kf 11/15/22 
    return acc

#Calculated in 'makeMassRes.py'
def massRes(mass):
    res = 1.06314 + 3.45955e-02*mass + -6.62113e-05*pow(mass,2) # 2016 simps kf 11/15/22
    return res

#Calculate efficiency vertex
def calculateTotalApProduction(mass_ap, eps, radFrac, radAcc, dNdm_CR):
    apProduction = (3.*(137./2.)*3.14159)* (mass_ap * eps*eps * radFrac * dNdm_CR)/radAcc
    return apProduction

def calculateExpectedSignal(mass, mass_ap, mass_pid, mass_lepton, fpid, alpha_dark, eps, selEffZ_h, target_pos, apProduction, signal_meanEnergyGeV, rho=True):
    ctau = simpeqs.getCtau(mass_ap, mass_pid, mass, eps, alpha_dark, fpid, mass_lepton, rho)
    gcTau = ctau*simpeqs.gamma(mass/1000.0, signal_meanEnergyGeV)
    effVtx = 0.0
    for z in range(len(selEffZ_h.axes[0].centers)):
        zz = selEffZ_h.axes[0].centers[z]
        if zz < target_pos:
            continue
        effVtx += (math.exp((target_pos - zz)/gcTau)/gcTau)*selEffZ_h.values()[z]
    br_VPi = 0.0
    if rho:
        br_VPi = simpeqs.br_Vrho_pi(mass_ap, mass_pid, mass, alpha_dark, fpid)
    else:
        br_VPi = simpeqs.br_Vphi_pi(mass_ap, mass_pid, mass, alpha_dark, fpid)
    br_V_ee = 1.0
    expSignal = apProduction * effVtx * br_VPi * br_V_ee
    return expSignal

## **CONFIGURE HISTOGRAMS** ##

In [ ]:

def defineHistos1d(category, color='blue'):
    histos = {}
    histos["unc_vtx_proj_sig"] = histo1d(f"unc_vtx_proj_sig_{category}",100,0,10,xlabel="V0 Projection Significance", label=category, color=color)
    histos["unc_vtx_deltaZ"] = histo1d(f"unc_vtx_deltaZ_{category}",50,0,10,xlabel="deltaZ", label=category, color=color)
    histos["unc_vtx_z"] = histo1d(f"unc_vtx_z_{category}",120,-20,100,xlabel="recon z [mm]", logY=True, label=category, color=color)
    histos["unc_vtx_zalpha_max"] = histo1d(f"unc_vtx_zalpha_max_{category}",1000,-10000,100,xlabel="zalpha max", logY=False, label=category, color=color)
    histos["vd_true_vtx_z"] = histo1d(f"vd_true_vtx_z_{category}",200,-50.3,149.7,xlabel="truth vtx z [mm]", logY=True, label=category, color=color)
    histos["vd_true_vtx_energy"] = histo1d(f"vd_true_vtx_energy_{category}",250,0.0,2.5,xlabel="truth energy [GeV]", logY=False, label=category, color=color)
    return histos

def fillVarHistos1d(histos_dict, array, reset=False):
    for key, histo in histos_dict.items():
        if reset == True:
            histo.reset()
        if key in array.fields:
            histo.fill(array[key])
            
def saveAnaHistos1d(outfile, histos_dict, mod='', subdir='', makePlots=False):
    for key, histo in histos_dict.items():
        writeHisto(outfile, histo,subdir=subdir)
        if makePlots:
            plotHisto1d(histo,mod=mod)
            makeStatsBox(histo)

In [ ]:
def histo1d(name,nbins,xmin,xmax,title='',label='',xlabel='',ylabel='events',logY=False, color='blue'):
    if title == '':
        title = name
    histogram = hist.Hist(hist.axis.Regular(nbins,xmin,xmax,flow=True, name=name,label=xlabel), 
          metadata={"name":name,"title":title, "xlabel":xlabel, "ylabel":ylabel, "label":label,
                   "logY":logY, "color":color})   
    return histogram

def makeStatsBox(h, x=0.5, y=0.9, spacing=0.05, color='red'):
    # Display number of entries, underflow, and overflow as text annotations
    plt.annotate(f'Entries: {h[::sum]}', xy=(x, y), xycoords='axes fraction', color=color)
    plt.annotate(f'Overflow: {h[hist.overflow]}', xy=(x, y-(1.*spacing)), xycoords='axes fraction', color=color)
    plt.annotate(f'Underflow: {h[hist.underflow]}', xy=(x, y-(2.*spacing)), xycoords='axes fraction', color=color)


def plotHisto1d(histo,mod=''):
    fig,ax = plt.subplots()
    histo.plot(label=histo.metadata.get("label"),color=histo.metadata.get("color"))
    plt.title("%s_%s"%(histo.metadata.get("title"),mod))
    plt.xlabel(histo.metadata.get("xlabel"))
    plt.ylabel(histo.metadata.get("ylabel"))
    if histo.metadata.get("logY") == True:
        plt.yscale('log')
    plt.legend()
    
def writeHisto(outfile, histo, subdir='', name=''):
    if name == '':
        name = histo.metadata.get("name")
    outfile[f'{subdir}/{name}'] = histo


## **Configure the run parameters**

In [ ]:
#Specify mixing
logeps2 = -5.5
eps2 = pow(10,logeps2)
eps = np.sqrt(eps2)

#Mass
mass = 65
mass_nsigma = 3.5
mass_low = float(mass) - mass_nsigma*massRes(float(mass))/2.0
mass_high = float(mass) + mass_nsigma*massRes(float(mass))/2.0
print(f'MASS WINDOW: {mass_low} - {mass_high} MeV')

#SIMP Parameters
alpha_dark = 0.01
mass_ratio_ap_to_vd = 1.66
mass_ratio_ap_to_pid = 3.0
ratio_mpi_to_fpi = 4.0*3.14159
mass_lepton = 0.511

#Get masses
mass_ap = mass*mass_ratio_ap_to_vd
mass_low_ap = mass_ratio_ap_to_vd*mass_low
mass_high_ap = mass_ratio_ap_to_vd*mass_high
mass_pid = mass_ap/mass_ratio_ap_to_pid
fpid = mass_pid/ratio_mpi_to_fpi

#RadFrac and RadAcc eval at Ap mass
radFrac = radiativeFraction(mass_ap) 
radAcc = radiativeAcceptance(mass_ap) 

#zalpha slope transformation
zalpha_slope = 0.01

## **CONFIGURE DATA AND MC** ##

In [ ]:
#Background and Signal Files
#truth_subdir_name = 'vtxana_radMatchTight_2016_simp_reach_SR'
#truth_tree_name = "vtxana_radMatchTight_2016_simp_reach_SR_tree"

#background/data
bkg_branches = ["unc_vtx_mass","unc_vtx_z","unc_vtx_proj_sig","unc_vtx_deltaZ","unc_vtx_ele_track_z0","unc_vtx_pos_track_z0","unc_vtx_ele_track_tanLambda"]
bkg_filename = '/sdf/group/hps/users/alspellm/projects/THESIS/data/2016/BLPass4c_20231006/ana_20231019/full_hadd_blpass4c_ana.root'
bkg_subdir = 'vtxana_Tight_2016_simp_reach_SR'
bkg_treename = 'vtxana_Tight_2016_simp_reach_SR_tree'
bkg_subdirCR = 'vtxana_Tight_2016_simp_reach_CR'
bkg_treenameCR = 'vtxana_Tight_2016_simp_reach_CR_tree'
bkg_branchesCR = ["unc_vtx_mass"]
bkg_sf_SR = 1.0
bkg_sf_CR = 1.0

#Signal
signal_branches = ["unc_vtx_mass","unc_vtx_z","unc_vtx_proj_sig","unc_vtx_deltaZ","unc_vtx_ele_track_z0","unc_vtx_pos_track_z0","unc_vtx_ele_track_tanLambda","unc_vtx_pos_track_tanLambda","vd_true_vtx_z","vd_true_vtx_energy"]
signal_subdir = 'vtxana_radMatchTight_2016_simp_reach_SR'
signal_treename = 'vtxana_radMatchTight_2016_simp_reach_SR_tree'
signal_filename = '/sdf/group/hps/users/alspellm/projects/THESIS/mc/2016/simps/signal_beam/20230713_slic/20230713_readout/hps-java_v5pt2pt1/pass4/recon_20231009/ana_20231020/hadd_simp_signal_%s_MeV_beam_ana.root'%(int(mass))
signal_sf = 10.0
#Truth Signal
signal_truth_filename = '/sdf/group/hps/users/alspellm/projects/THESIS/mc/2016/simps/slic/20230713_slic/20230724_slic_ana/ana_files/hadd_simp_%s_MeV_rot_slic_mcana.root'%(int(mass))
truth_mc_selection = 'vtxana_mc_radMatchTight_2016_simp_reach_SR'
truth_mc_tree='vtxana_mc_radMatchTight_2016_simp_reach_SR_tree'
pdgid = 625

## **LOAD TRUTH SIGNAL** ##

In [ ]:
#Signal truth recon z
sbudir = "vtxana_radMatchTight_2016_simp_reach_SR"
signal_truth_file = uproot.open(signal_truth_filename)
#truth_z_h = signal_truth_file[f'{subdir}/{tree_name}'].arrays(branches=[f'vdSimZ
truth_z_h = signal_truth_file["mcAna/mcAna_mc625Z_h"].to_hist()
signal_truth_file.close
signal_truthE_h = histo1d('signal_truthE',250,0.0,2.5,title='truth energy',label='truth energy',xlabel='truth energy [GeV]',ylabel='MC Events',logY=False, color='blue')
simZ_h = histo1d('signal_simZ',200,-50.3,149.7,title='signal_simZ',label='signal simZ',xlabel='true zvtx [mm]',ylabel='MC Events',logY=False, color='blue')
#selZ_h = histo1d('signal_selZ',200,-50.3,149.7,title='signal_selZ',label='signal selZ',xlabel='true zvtx [mm]',ylabel='MC Events',logY=False, color='blue')
simZ_h.fill(truth_z_h.axes[0].centers, weight=truth_z_h.counts())

## **LOAD DATA** ##

In [ ]:
#Read background for mass window
bkg_file = uproot.open(bkg_filename)
bkg_tree = bkg_file[f'{bkg_subdir}/{bkg_treename}']
bkg_arrays = getTBranchArrays(bkg_tree,bkg_branches, mass_low, mass_high)
bkg_treeCR = bkg_file[f'{bkg_subdirCR}/{bkg_treenameCR}']

#Get background rate in control region, eval at Ap mass, NOT Vd mass
bkg_arraysCR = getTBranchArrays(bkg_treeCR, bkg_branchesCR, 
                                mass_low_ap, mass_high_ap)

In [ ]:
dNdm = bkg_sf_CR*len(bkg_arraysCR["unc_vtx_mass"])/(mass_high_ap-mass_low_ap)
#dNdm = dNdm_h.counts().sum()/(mass_high-mass_low)
print("Background Rate dNdm:",dNdm)

bkg_file.close()

#Add zalpha max
#max_zalpha_arr = np.maximum(zalphaTransformation(zalpha_slope,bkg_arrays["unc_vtx_z"],bkg_arrays["unc_vtx_ele_track_z0"]),
                 #           zalphaTransformation(zalpha_slope,bkg_arrays["unc_vtx_z"],bkg_arrays["unc_vtx_pos_track_z0"]))
#bkg_arrays["unc_vtx_zalpha_max"] = max_zalpha_arr

## **LOAD SIGNAL** ##

In [ ]:
#Read signal for mass window
signal_file = uproot.open(signal_filename)
signal_tree = signal_file[f'{signal_subdir}/{signal_treename}']
signal_arrays = getTBranchArrays(signal_tree,signal_branches,mass_low,mass_high)
signal_file.close()

#Apply z0 corrections
applyCorrection(signal_arrays,'unc_vtx_ele_track_z0', -0.058)
applyCorrection(signal_arrays,'unc_vtx_pos_track_z0', -0.098)

#Add zalpha max
#max_zalpha_arr = np.maximum(zalphaTransformation(zalpha_slope,signal_arrays["unc_vtx_z"],signal_arrays["unc_vtx_ele_track_z0"]),
#                            zalphaTransformation(zalpha_slope,signal_arrays["unc_vtx_z"],signal_arrays["unc_vtx_pos_track_z0"]))
#signal_arrays["unc_vtx_zalpha_max"] = max_zalpha_arr

In [ ]:
## **INITIALIZE HISTOGRAMS** ##

In [ ]:
#Create ROOT file to store histograms
outdir = 'output_larger_slopes'
outfile = uproot.recreate(f'./{outdir}/mass_{mass}_zalpha_slope_study.root')
#Define Initial background and signal histograms
bkg_histos1d = defineHistos1d('bkg') 
signal_histos1d = defineHistos1d('signal', color='red')
#Fill initial histograms
fillVarHistos1d(bkg_histos1d, bkg_arrays)
fillVarHistos1d(signal_histos1d, signal_arrays)

mod = 'initial'
saveAnaHistos1d(outfile, bkg_histos1d, subdir=mod, mod=mod, makePlots=False)
saveAnaHistos1d(outfile, signal_histos1d, subdir=mod, mod=mod, makePlots=False)

## **Configure High-Z Cuts** ##

In [ ]:
deltaZ_par0 = 18.5972
deltaZ_par1 = 0.159555
v0projSig_cutvalue = 2.0

In [ ]:
#Apply initial high-z cuts
#v0ProjSigCut(signal_arrays, v0projSig_cutvalue)
bkg_arrays = v0ProjSigCut(bkg_arrays, v0projSig_cutvalue)
bkg_arrays = deltaZCut(bkg_arrays, deltaZ_par0, deltaZ_par1, mass)

#Apply initial high-z cuts
signal_arrays = v0ProjSigCut(signal_arrays, v0projSig_cutvalue)
signal_arrays = deltaZCut(signal_arrays, deltaZ_par0, deltaZ_par1, mass)

## **BEGIN ITERATING CUT VALUE** ##

In [ ]:
def findCutValue(initial_histo, cut_fraction):
    initial_integral = initial_histo[::sum].value
    test_int = 0.0
    start_bin = initial_histo.shape[0]-1
    while test_int < (cut_fraction)*initial_integral:
        test_int = initial_histo[start_bin:hist.overflow:sum].value
        if test_int >= (cut_fraction)*initial_integral:
            break
        else:
            start_bin = start_bin - 1
        if start_bin == 0:
            break
    return initial_histo.axes[0].centers[start_bin]
            
#Iterate over zalpha slopes
from matplotlib.backends.backend_pdf import PdfPages
n_slopes = 6
figures = {}
for n,zalpha_slope in enumerate(range(1,n_slopes+1)):
    zalpha_slope = zalpha_slope/10000.
    zalpha_subdir = f'slope_{zalpha_slope}'
    print("Running Zalpha Slope:",zalpha_slope)
    
    #Copy arrays
    iter_signal_arrays = signal_arrays
    iter_bkg_arrays = bkg_arrays

    #Add signal zalpha max
    max_zalpha_arr_sig = np.maximum(zalphaTransformation(zalpha_slope,iter_signal_arrays["unc_vtx_z"],iter_signal_arrays["unc_vtx_ele_track_z0"]),
                                zalphaTransformation(zalpha_slope,iter_signal_arrays["unc_vtx_z"],iter_signal_arrays["unc_vtx_pos_track_z0"]))
    iter_signal_arrays["unc_vtx_zalpha_max"] = max_zalpha_arr_sig

    #Add bkg zalpha max
    max_zalpha_arr_bkg = np.maximum(zalphaTransformation(zalpha_slope,iter_bkg_arrays["unc_vtx_z"],iter_bkg_arrays["unc_vtx_ele_track_z0"]),
                                zalphaTransformation(zalpha_slope,iter_bkg_arrays["unc_vtx_z"],iter_bkg_arrays["unc_vtx_pos_track_z0"]))
    iter_bkg_arrays["unc_vtx_zalpha_max"] = max_zalpha_arr_bkg
    
    minzalpha = ak.min(ak.concatenate([max_zalpha_arr_sig, max_zalpha_arr_bkg]))-10.0
    #maxzalpha = ak.max(ak.concatenate([max_zalpha_arr_sig, max_zalpha_arr_bkg]))+10.0
    maxzalpha = 100
    #nbins = int((maxzalpha-minzalpha)/10.0)
    nbins = 1000
    
    signal_histos1d["unc_vtx_zalpha_max"] = histo1d(f"unc_vtx_zalpha_max_signal",nbins,minzalpha,maxzalpha,xlabel="zalpha max", logY=False, label='signal')
    bkg_histos1d["unc_vtx_zalpha_max"] = histo1d(f"unc_vtx_zalpha_max_bkg",nbins,minzalpha,maxzalpha,xlabel="zalpha max", logY=False, label='bkg')
    
    #Fill histograms before iterating
    fillVarHistos1d(bkg_histos1d, iter_bkg_arrays,reset=True)
    fillVarHistos1d(signal_histos1d, iter_signal_arrays, reset=True)
    mod = 'pre_iter'
    saveAnaHistos1d(outfile, bkg_histos1d, subdir=f'{zalpha_subdir}/{mod}', mod=mod,makePlots=False)
    saveAnaHistos1d(outfile, signal_histos1d, subdir=f'{zalpha_subdir}/{mod}', mod=mod, makePlots=False)

    #Graphs to track performance
    nsigs = []
    nbkgs = []
    zbis = []
    cut_values = []
    effs = []

    max_iter = 95
    step_size = 0.01
    initial_histo = outfile[f'{zalpha_subdir}/pre_iter/unc_vtx_zalpha_max_signal'].to_hist()
    #initial_nsig = (outfile["initial/unc_vtx_zalpha_max_signal"].to_hist()[::sum].value)
    for iteration in range(0,max_iter):
        if iteration%10 == 0:
            print("Iteration: ", iteration)
        iter_subdir=f'{zalpha_subdir}/iter_{iteration}'
        cut_fraction = iteration*step_size
        #cut_value = findCutValue(signal_histos1d["unc_vtx_zalpha_max"], initial_nsig, iteration, step_size)
        cut_value = findCutValue(initial_histo, cut_fraction)
        cut_values.append(cut_value)
        #print("cut value:",cut_value)
        #Apply cut to signal and background
        if iteration > 0:
            iter_signal_arrays = zalphaCut(iter_signal_arrays,cut_value)
            iter_bkg_arrays = zalphaCut(iter_bkg_arrays,cut_value)
        
        #Signal efficiency starting from no cut
        eff = len(iter_signal_arrays['unc_vtx_zalpha_max'])/initial_histo[::sum].value
        effs.append(eff)

        #Fill histograms for this iteration
        mod = f'iter_{iteration}_slope_{zalpha_slope}'
        fillVarHistos1d(bkg_histos1d, iter_bkg_arrays,reset=True)
        fillVarHistos1d(signal_histos1d, iter_signal_arrays, reset=True)
        saveAnaHistos1d(outfile, bkg_histos1d, subdir = iter_subdir, mod=mod,makePlots=False)
        saveAnaHistos1d(outfile, signal_histos1d, subdir = iter_subdir, mod=mod,makePlots=False)

        #Count remaining background beyond zposition
        target_pos = -4.3 #mm
        nbkg = bkg_sf_SR*bkg_histos1d["unc_vtx_z"][loc(target_pos):hist.overflow:sum]
        nbkgs.append(nbkg)
        #print("Nbkg:",nbkg)

        #Get signal selection efficiency F(z)
        weights=signal_histos1d["vd_true_vtx_z"].counts()/simZ_h.counts()
        nan_mask = np.isnan(weights)
        weights = ak.where(nan_mask, 0, weights)
        selEffZ_h = histo1d(f"sel_eff_signal",200,-50.3,149.7,xlabel="truth vtx z [mm]", logY=True, label='signal', color='green')
        selEffZ_h.fill(signal_histos1d["vd_true_vtx_z"].axes[0].centers, 
                       weight=weights)
        writeHisto(outfile,selEffZ_h, subdir=iter_subdir)

        #Calculate Expected Signal
        signal_meanEnergyGeV = 1.4

        apProduction = calculateTotalApProduction(mass_ap, eps, radFrac, radAcc, dNdm)
        nsig_rho = calculateExpectedSignal(mass, mass_ap, mass_pid, mass_lepton, fpid, alpha_dark, eps, selEffZ_h, target_pos, apProduction, signal_meanEnergyGeV, True)
        nsig_phi = calculateExpectedSignal(mass, mass_ap, mass_pid, mass_lepton, fpid, alpha_dark, eps, selEffZ_h, target_pos, apProduction, signal_meanEnergyGeV, False)
        #print("Expected Signal rho:",nsig_rho)
        #print("Expected Signal phi:",nsig_phi)
        nsig_total = signal_sf*(nsig_rho+nsig_phi)
        nsigs.append(nsig_total)
        #print("Total Expected Signal:", nsig_total)
        zbi = calculateZBi(nsig_total+nbkg, nbkg, 1.0)
        zbis.append(zbi)
    
    
    
    xvalues = [i+1 for i in range(len(zbis))]
    #fig, ax = plt.subplots()
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(xvalues, nsigs, label='Nsig',color='blue')
    ax.plot(xvalues, nbkgs, label='Nbkg',color='red')
    ax.set_yscale('log')
    ax.set_ylabel('Nsig and Nbkg')
    ax.set_xlabel("Iteration Number")
    ax.tick_params(axis='y')
    ax.set_ylim(bottom=0.0, top=1000)
    ax.yaxis.grid(True, linestyle='--', which='major', color='gray', alpha=0.7)
    
    ax2 = ax.twinx()
    ax2.plot(xvalues, zbis, label='ZBi',color='green')
    ax2.set_ylabel('ZBi', color='green')
    ax2.tick_params(axis='y',labelcolor='green')
    ax2.set_ylim(bottom=0.1, top=8)
    ax2.text(1,6,f'zalpha_slope_{zalpha_slope}')

    fig.legend(loc='center')
    figures[f'zalpha_slope_{zalpha_slope}'] = fig
    
    #Save data as TGraphs
    graphs = {}
    graphs['nsig'] = r.TGraph(len(xvalues), np.array(xvalues,dtype=float), np.array(nsigs,dtype=float))
    graphs['nbkg'] = r.TGraph(len(xvalues), np.array(xvalues,dtype=float), np.array(nbkgs,dtype=float))
    graphs['zbis'] = r.TGraph(len(xvalues), np.array(xvalues,dtype=float), np.array(zbis,dtype=float))
    graphs['cut_values'] = r.TGraph(len(xvalues), np.array(xvalues,dtype=float), np.array(cut_values,dtype=float))
    graphs['eff'] = r.TGraph(len(xvalues), np.array(xvalues,dtype=float), np.array(cut_values,dtype=float))
    for key, graph in graphs.items():
        outfile[f'{zalpha_subdir}/{key}_g'] = graph
    
    

In [18]:
with PdfPages(f'{outdir}/mass_{mass}_zalpha_slope_plots.pdf') as pdf:
    for fig in figures.values():
        fig.axes[0].set_ylim(bottom=1,top=500000)
        pdf.savefig(fig)
    

'texgyreheros-regular.otf' can not be subsetted into a Type 3 font. The entire font will be embedded in the output.


In [ ]:
    def makeGraph(x,y,label='',xlabel='',ylabel='',title='',color='blue',logy=False):
        #fig = plt.figure()
        #ax = fig.add_subplot()
        #plot = ax.plot(x, y, label=label, color=color)[0]
        fig, ax = plt.subplots()
        plot = ax.plot(x, y, label=label, color=color)[0]
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.legend()
        if logy:
            ax.set_yscale('log')
        return plot


    #Graph results
    xvalues = [i+1 for i in range(len(zbis))]
    nsig_plot = makeGraph(xvalues, nsigs, label='Nsig', xlabel='Iteration', ylabel='Nsig',title='Expected Signal', color='blue')
    nbkg_plot = makeGraph(xvalues, nbkgs, label='Nbkg', xlabel='Iteration', ylabel='Nbkg',title='Total Bkg Beyond Target in SR', color='red',logy=True)
    zbi_plot = makeGraph(xvalues, zbis, label='ZBi', xlabel='Iteration', ylabel='ZBi',title='ZBi Significance', color='green')
    #nsig_fig,nsig_ax = makeGraph(xvalues, nsigs, label='Nsig', xlabel='Iteration', ylabel='Nsig',title='Expected Signal', color='blue')
    #nbkg_fig,nbkg_ax = makeGraph(xvalues, nbkgs, label='Nbkg', xlabel='Iteration', ylabel='Nbkg',title='Total Bkg Beyond Target in SR', color='red')
    #zbi_fig,zbi_ax = makeGraph(xvalues, zbis, label='ZBi', xlabel='Iteration', ylabel='ZBi',title='ZBi Significance', color='green')

In [ ]:
    '''
    #overlayGraphs([zbi_plot, nsig_plot,nbkg_plot])
    fig, ax = plt.subplots()
    ax.plot(nsig_plot.get_xdata(), nsig_plot.get_ydata(), label=nsig_plot.get_label(),color=nsig_plot.get_color())
    ax.set_yscale('log')
    ax.set_ylabel('Nsig and Nbkg', color=nsig_plot.get_color())
    ax.plot(nbkg_plot.get_xdata(), nbkg_plot.get_ydata(), label=nbkg_plot.get_label(),color=nbkg_plot.get_color())
    ax.tick_params(axis='y',labelcolor=nsig_plot.get_color())
    ax.set_ylim(bottom=0.0, top=500)
    #ax.yaxis.set_major_locator(plt.MaxNLocator(nbins=20))
    ax.yaxis.grid(True, linestyle='--', which='major', color='gray', alpha=0.7)

    ax2=ax.twinx()
    #ax2.set_yscale('log')
    ax2.set_ylabel('ZBi', color=zbi_plot.get_color())
    ax2.tick_params(axis='y',labelcolor=zbi_plot.get_color())
    ax2.set_ylim(bottom=0.1, top=7)
    ax2.plot(zbi_plot.get_xdata(), zbi_plot.get_ydata(), label=zbi_plot.get_label(),color=zbi_plot.get_color())

    fig.legend(loc='center')
    
    fig.savefig(f'figures/zalpha_slope_{str(zalpha_slope)}_results.png')
    final_figures[f'{zalpha_slope}'] = 
    '''
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.plot(nsig_plot.get_xdata(), nsig_plot.get_ydata(), label=nsig_plot.get_label(),color=nsig_plot.get_color())
    ax.set_yscale('log')
    ax.set_ylabel('Nsig and Nbkg', color=nsig_plot.get_color())
    ax.plot(nbkg_plot.get_xdata(), nbkg_plot.get_ydata(), label=nbkg_plot.get_label(),color=nbkg_plot.get_color())
    ax.tick_params(axis='y',labelcolor=nsig_plot.get_color())
    ax.set_ylim(bottom=0.0, top=500)
    #ax.yaxis.set_major_locator(plt.MaxNLocator(nbins=20))
    ax.yaxis.grid(True, linestyle='--', which='major', color='gray', alpha=0.7)

    ax2=ax.twinx()
    #ax2.set_yscale('log')
    ax2.set_ylabel('ZBi', color=zbi_plot.get_color())
    ax2.tick_params(axis='y',labelcolor=zbi_plot.get_color())
    ax2.set_ylim(bottom=0.1, top=7)
    ax2.plot(zbi_plot.get_xdata(), zbi_plot.get_ydata(), label=zbi_plot.get_label(),color=zbi_plot.get_color())

    fig.legend(loc='center')
    
    fig.savefig(f'figures/zalpha_slope_{str(zalpha_slope)}_results.png')
    final_figures[f'{zalpha_slope}'] = fig

print("done")

#Check cuts applied correctly
print(signal_arrays["unc_vtx_proj_sig"])
v0projsig_sig_h = Hist(
    hist.axis.Regular(100,0,10, name="unc_vtx_proj_sig_signal", label="V0 Projection Significance")
)
v0projsig_sig_h.fill(signal_arrays["unc_vtx_proj_sig"])

deltaZ_sig_h = Hist(
    hist.axis.Regular(50,0,10, name="unc_vtx_deltaZ_signal", label="deltaZ")
)
deltaZ_sig_h.fill(signal_arrays["unc_vtx_deltaZ"])

fig, ax = plt.subplots()
v0projsig_sig_h.plot()
plt.show()

fig, ax = plt.subplots()
deltaZ_sig_h.plot()
plt.show()

## **Relevant Files Have Been Loaded Above** ##

## **Check Z0 parameters** ##

#Check parameter distributions
impact_param_sig_hh = Hist(
    hist.axis.Regular(100,-20,80, name="recon_z",label="vtx z [mm]"),
    hist.axis.Regular(200,-5,5, name="track_z0",label="track z0 [mm]")
)
impact_param_sig_hh.fill(signal_arrays["unc_vtx_z"], signal_arrays["unc_vtx_ele_track_z0"]) 
impact_param_sig_hh.fill(signal_arrays["unc_vtx_z"], signal_arrays["unc_vtx_pos_track_z0"]) 
fig, ax = plt.subplots()
impact_param_sig_hh.plot(cmap=cmap)
plt.show()

#ele track z0
ele_track_z0_h = Hist(
    hist.axis.Regular(100,-5,5, name="ele_track_z0_h", label="ele track z0 [mm]")
)
ele_track_z0_h.fill(signal_arrays["unc_vtx_ele_track_z0"])
mean = np.sum(ele_track_z0_h.axes[0].centers*ele_track_z0_h.values())/ele_track_z0_h.sum()
print('ele track z0 mean:',mean)
#ele track z0 correction = -0.58

#pos track z0
pos_track_z0_h = Hist(
    hist.axis.Regular(100,-5,5, name="pos_track_z0_h", label="pos track z0 [mm]")
)
pos_track_z0_h.fill(signal_arrays["unc_vtx_pos_track_z0"])
mean = np.sum(pos_track_z0_h.axes[0].centers*pos_track_z0_h.values())/pos_track_z0_h.sum()
print('pos track z0 mean:',mean)
#pos track z0 correction = -0.98
fig, ax = plt.subplots()
ele_track_z0_h.plot(label='ele')
pos_track_z0_h.plot(label='pos')
plt.legend()
plt.show()

#All track z0
track_z0_h = Hist(
    hist.axis.Regular(100,-5,5, name="track_z0_h", label="track z0 [mm]")
)
track_z0_h.fill(signal_arrays["unc_vtx_ele_track_z0"])
track_z0_h.fill(signal_arrays["unc_vtx_pos_track_z0"])
track_z0_h.plot()
mean = np.sum(track_z0_h.axes[0].centers*track_z0_h.values())/track_z0_h.sum()
print('trackz z0 mean:',mean)
plt.show()

#Check z0/tanlambda
z0tanlambda_sig_hh = Hist(
    hist.axis.Regular(400,-150,50, name="z0/tanlambda",label="z0/tanlambda"),
    hist.axis.Regular(180,-20,70, name="recon_z",label="vtx z [mm]")
)
z0tanlambda_sig_hh.fill(signal_arrays["unc_vtx_ele_track_z0"]/signal_arrays["unc_vtx_ele_track_tanLambda"],signal_arrays["unc_vtx_z"])
z0tanlambda_sig_hh.fill(signal_arrays["unc_vtx_pos_track_z0"]/signal_arrays["unc_vtx_pos_track_tanLambda"],signal_arrays["unc_vtx_z"])
z0tanlambda_sig_hh.plot(cmap=cmap)
plt.show()


## **Create Zalpha and add to tree** ##

slope = 0.01
def zalphaTransformation(slope, recon_z, z0):
    condition = z0 > 0.0
    zalpha = ak.where(condition, recon_z - (z0/slope), recon_z + (z0/slope))
    return zalpha
max_zalpha_arr = np.maximum(zalphaTransformation(slope,signal_arrays["unc_vtx_z"],signal_arrays["unc_vtx_ele_track_z0"]),
                            zalphaTransformation(slope,signal_arrays["unc_vtx_z"],signal_arrays["unc_vtx_pos_track_z0"]))
signal_arrays["unc_vtx_zalpha_max"] = max_zalpha_arr
signal_arrays.fields
#Plot zalpha 
zalpha_sig_hh = Hist(
    hist.axis.Regular(200,-20,80, name="recon_z",label="vtx z [mm]"),
    hist.axis.Regular(1200,-300,100, name="zalpha_max",label=f'zalpha({slope}) max')
)
zalpha_sig_hh.fill(signal_arrays["unc_vtx_z"], signal_arrays["unc_vtx_zalpha_max"])
fig, ax = plt.subplots()
zalpha_sig_hh.plot(cmap=cmap)
plt.show()


h = hist.Hist.new.Reg(50,0,20,name="V0ProjSig",label="V0ProjSig Nsigma").Weight()
h.fill(signal_arrays["unc_vtx_proj_sig"])
plt.ylabel("Events")
plt.xlabel("V0ProjSig")
h.plot()

#for branch in tree_branches:
#    signal_tree[branch].to_hist()